In [9]:
#from comet_ml import Experiment
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [10]:
DATA_PATH = "/Users/tristanmartin/Desktop/UdeM_PhD/Cours/A2023/IFT6758/Project/Milestone_1/data"
PROJECT_PATH = "/Users/tristanmartin/Desktop/UdeM_PhD/Cours/A2023/IFT6758/Project/IFT6758B-Project-B10-main-2/Milestone2/"

#DATA_PATH = '../../IFT6758_Data/'
#PROJECT_PATH = '../../Milestone2/'

import sys
sys.path.append(PROJECT_PATH)

from features.feature_eng2 import *
from features.tidy_data import *

In [ ]:
# with open('../../COMET_API_KEY.txt', 'r') as f:
#     my_key = f.read()

# experiment = Experiment(
#   api_key=my_key,
#   project_name="nhl-project-b10",
#   workspace="ift6758b-project-b10"
# )

In [11]:
#get_train_data(DATA_PATH)

In [12]:
# Loading data and pre-processing
X = pd.read_csv(DATA_PATH + '/train_data.csv', index_col=0)
has_nan = X.isna().any().any()

if has_nan:
    print("There are NaN values in the DataFrame 'X'.")
    # Dropping NaNs since these events do not have x and y coordinates
    X.dropna(inplace=True)
    X = X.reset_index(drop=True)
else:
    print("There are no NaN values in the DataFrame 'X'.")

X = X[~X.isin([np.nan, np.inf, -np.inf]).any(axis = 1)]
X = X.reset_index(drop=True)
num_cols = X.select_dtypes([np.number]).columns
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

categorical_cols = X.select_dtypes(exclude=["number", "bool"]).columns
X = pd.get_dummies(data=X, columns=categorical_cols)

boolean_cols = X.select_dtypes([bool]).columns
X[boolean_cols] = X[boolean_cols].astype(int)
X = X.reset_index(drop=True)

There are NaN values in the DataFrame 'X'.


In [13]:
# Target
y = X['is_goal']
# Feature DataFrame
X = X.drop('is_goal', axis=1)

#N = 1000
#y = y.head(N)
#X = X.head(N)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Max_feature adds REGULARIZATION!
RandomForest_model = RandomForestRegressor(n_estimators=10, max_features='sqrt', random_state=0)
#rf_regressor = RandomForestRegressor(n_estimators=10, random_state=0)
sel = RFECV(estimator=RandomForest_model, step=1, cv=5, scoring='neg_mean_squared_error')
X_train_sel = sel.fit_transform(X_train, y_train)
X_test_sel = sel.transform(X_test)

# Training and predicting with cleaned dataset
RandomForest_model.fit(X_train_sel, y_train)
y_pred = RandomForest_model.predict(X_test_sel)

mse = metrics.mean_squared_error(y_test, y_pred)
print("Mean MSE with RFECV Feature Selection:", mse)

Mean MSE with RFECV Feature Selection: 0.9416102396044391
R-squared on Test Set: -0.5093795093795093
